<img src="https://dutchanalytics.com/wp-content/uploads/2020/02/svg_logo_dutch_analytics-1.svg" width="80">

# Quickstart MNIST demo
Deploy your model in UbiOps using the python client library.


The first step is to download a prepared model and some sample data.

In [ ]:
!curl -X GET https://storage.googleapis.com/ubiops/example-model-packages/mnist_model_package.zip -o mnist_model_package.zip
!curl -X GET https://storage.googleapis.com/ubiops/example-model-packages/1.jpg -o 1.jpg
!curl -X GET https://storage.googleapis.com/ubiops/example-model-packages/2.jpg -o 2.jpg
!curl -X GET https://storage.googleapis.com/ubiops/example-model-packages/3.jpg -o 3.jpg

Add your API token, provide a project name, model name and model version name. 
Afterward initialize the client library.

In [ ]:
API_TOKEN = 'Token <YOUR API_KEY>'
PROJECT_NAME = '<YOUR PROJECT NAME>'
MODEL_NAME = 'mnist-tutorial'
MODEL_VERSION = 'v1'
image_files = ['1.jpg', '2.jpg', '3.jpg']

In [ ]:
import ubiops
configuration = ubiops.Configuration()
configuration.api_key['Authorization'] = API_TOKEN
client = ubiops.ApiClient(configuration)
api = ubiops.CoreApi(client)
api.service_status()

## Deploy the model
Create a model for mnist.

In [ ]:
mnist_model_template = ubiops.ModelCreate(
    name=f'{MODEL_NAME}',
    description='A model to classify handwritten digits.',
    input_type='structured',
    output_type='structured',
    input_fields=[
        ubiops.ModelInputFieldCreate(
            name='image',
            data_type='blob',
        )
    ],
    output_fields=[
        ubiops.ModelOutputFieldCreate(
            name='prediction',
            data_type='int',
        ),
        ubiops.ModelOutputFieldCreate(
            name='probability',
            data_type='double'
        )
    ]
)

mnist_model = api.models_create(
    project_name=f'{PROJECT_NAME}',
    data=mnist_model_template
)
print(mnist_model)

Create a model version for mnist.

In [ ]:
mnist_model_version_template = ubiops.ModelVersionCreate(
    version=f'{MODEL_VERSION}',
    language='python3.6',
    memory_allocation=1024,
    maximum_instances=1,
    minimum_instances=0,
    maximum_idle_time=1800 # = 30 minutes
)

mnist_model_version = api.model_versions_create(
    project_name=f'{PROJECT_NAME}',
    model_name=f'{MODEL_NAME}',
    data=mnist_model_version_template
)
print(mnist_model_version)

Upload the prepared model file to the created model version.

In [ ]:
file_upload_result =api.model_versions_file_upload(
    project_name=f'{PROJECT_NAME}',
    model_name=f'{MODEL_NAME}',
    version=f'{MODEL_VERSION}',
    file='mnist_model_package.zip'
)

Check if model version is finished building.
This can take a few minutes.

In [ ]:
from time import sleep
status = 'building'
while status != 'available' and 'failed' not in status:    
    model_version_status = api.model_versions_get(       
        project_name=f'{PROJECT_NAME}',        
        model_name=f'{MODEL_NAME}',        
        version=f'{MODEL_VERSION}'    
    )    
    status = model_version_status.status
    sleep(1)
print(status)

## Create multiple direct requests
Make sure the model is in 'available' state before performing model requests.

Create a model request for each image and get the results directly if the request is finished.

In [ ]:
from IPython.display import Image, display

for image_file in image_files:
    # Display the input image
    display(Image(url=image_file, width=40, height=40))
    
    # First upload the image
    blob = api.blobs_create(project_name=f'{PROJECT_NAME}', file=image_file)
    
    # Make a request using the blob id as input.
    data = {'image': blob.id}
    model_result = api.model_requests_create(
        project_name=f'{PROJECT_NAME}',
        model_name=f'{MODEL_NAME}',
        version=f'{MODEL_VERSION}',
        data=data
    )
    print(model_result)

## Create single batch request
Make sure the model is in 'available' state before performing model requests. <br/>
Create a model batch request for all images at ones. This will create multiple asynchronuous requests, which means that the requests are queued in the back-end and the results can be collected at a later time.

In [ ]:
from IPython.display import Image, display

# Upload the images
blobs = []
for image_file in image_files:
    blobs.append(api.blobs_create(project_name=f'{PROJECT_NAME}', file=image_file))
    
# Make a batch request using the blob ids as input.
data = [{'image': blob.id} for blob in blobs]
created_batch = api.batch_model_requests_create(
    project_name=f'{PROJECT_NAME}',
    model_name=f'{MODEL_NAME}',
    version=f'{MODEL_VERSION}',
    data=data
)

# Display
for image_file, request in zip(image_files, created_batch):
    display(Image(url=image_file, width=40, height=40))
    print(request)

## Wait for results
Display request results when they are finished.

In [ ]:
from time import sleep

for request in created_batch:
    print(request.id, '', end='\r')
    
    status = 'pending'
    while status != 'completed' and 'failed' not in status:    
        request_response = api.batch_model_requests_get(       
            project_name=f'{PROJECT_NAME}',        
            model_name=f'{MODEL_NAME}',        
            version=f'{MODEL_VERSION}',
            request_id=request.id
        )    
        status = request_response.status
        sleep(1)
    print(request.id, status, request_response.result)

## List all batch requests
List all batch requests of the model. <br/>
Note that batch requests are stored for a limited time. See your subscription for more information.

In [ ]:
import pprint

all_batch_requests = api.batch_model_requests_list(
    project_name=f'{PROJECT_NAME}',
    model_name=f'{MODEL_NAME}',
    version=f'{MODEL_VERSION}'
)
pprint.pprint(all_batch_requests)

## Cleanup
Delete created model.

In [ ]:
api.models_delete(
    project_name=f'{PROJECT_NAME}',
    model_name=f'{MODEL_NAME}'
)